# Support Vector Machine

1. **SVM** performs well in high-dimensional spaces (e.g., text classification or image recognition) due to its ability to construct hyperplanes in such spaces effectively.
2. **SVM** can handle non-linear separable data effectively using kernel tricks (e.g., polynomial, radial basis function (RBF)).
3. **SVM** with a soft margin or appropriate kernel is more robust to **outliers** because it focuses on maximizing the margin and ignores points far away from the margin.

### Fields of sage of SVM 
- **Text Classification**:
SVM can be used for classifying text data into positive and negative categories, such as emails or product reviews.

- **Bioinformatics**:
SVM can be applied to protein classification, gene recognition, and predicting the properties of biomolecules.

- **Computer Vision**:
SVM can be utilized for object detection in images, face recognition, image classification, and more.

- **Financial Analysis**:
SVM can be employed to predict stock prices and classify investment products.

- **Internet Advertising**:
SVM can be used to classify users based on their online behavior and predict which advertisements will be most effective for specific users.

- **Data Analysis**:
SVM can be applied for clustering and classifying data such as customer data, sales data, and manufacturing process data.

- **Medical Diagnosis**:
SVM can be used to classify medical data, such as CT and MRI images, to determine the presence or absence of certain diseases.

In [5]:
import numpy as np
import pandas as pd

# Support Vector Machine
$$ y 
pred
​
 =sign(w⋅x+b) $$

## Loss Function
$$ L=0.5⋅∥w∥ 
2
 +C 
i=1
∑
n
​
 max(0,1−y 
i
​
 (w⋅x 
i
​
 +b)) $$


## for misclassification (w)
$$ w=w−η⋅(2w−C⋅y 
i
​
 ⋅x 
i
​
 ) $$

## Correct one w
$$ w=w−η⋅(2w) $$

## For Bias
$$ b=b−η⋅(−C⋅y 
i
​
 ) $$


### SVM with linear relationship

In [6]:
class SVC:
    def __init__(self, learning_rate, epochs) -> None:
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.weights = None
        self.bias = None
    
    def predict(self, X):
        return (np.dot(X, self.weights) + self.bias) > 0

    def fit(self, X, y, C=1):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.epochs):
            for i in range(n_samples):

                y_i = 2 * y[i] - 1

                if y_i * (np.dot(X[i], self.weights) + self.bias) < 1:
                    self.weights -= self.learning_rate * (2 * self.weights - C * y_i * X[i])
                    self.bias -= self.learning_rate * (-C * y_i)
                else:
                    self.weights -= self.learning_rate * (2 * self.weights)
        return self.weights, self.bias

In [7]:
np.random.seed(42)
n = 50


X1 = np.random.uniform(0, 10, n)  
X2 = np.random.uniform(0, 10, n)  

z = 2 * X1 - 1.5 * X2 - 5
probabilities = 1 / (1 + np.exp(-z))
y = (probabilities > 0.5).astype(int)  

df = pd.DataFrame({
    "Feature1": X1,
    "Feature2": X2,
    "Target": y
})

X = df.drop("Target", axis=1).to_numpy()
y = df["Target"].to_numpy()

In [8]:
svm = SVC(learning_rate=0.01, epochs=1000)
svm.fit(X, y)
y_pred = (svm.predict(X) >= 0.5).astype(int)
np.mean(y_pred == y)

0.94

### SVM with kernel-trick for non-linear relationship

In [9]:
from sklearn.datasets import make_circles
from sklearn.model_selection import train_test_split

X, y = make_circles(n_samples=2000, noise=0.1, factor=0.3, random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix


svm = SVC()
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
pr_score = precision_score(y_test, y_pred)
rc_score = recall_score(y_test, y_pred)
f_score = f1_score(y_test, y_pred)
cf_matrix = confusion_matrix(y_test, y_pred)

print("Accuracy: ", accuracy)
print("Persition score: ", pr_score)
print("Recall score: ", rc_score)
print("F1 Score: ", f_score)
print("-----------------------------")
print("Confusion Matrix",cf_matrix)

Accuracy:  1.0
Persition score:  1.0
Recall score:  1.0
F1 Score:  1.0
-----------------------------
Confusion Matrix [[197   0]
 [  0 203]]


In [11]:
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

churn_df = pd.read_csv("./Telco-Customer-Churn.csv").rename(columns={"Unnamed: 0":"customerID"})


churn_df = churn_df.drop(["customerID", "TotalCharges"], axis=1)
churn_df["Churn"] = churn_df["Churn"].apply(lambda x: 1 if x=="Yes" else 0)

non_numeric_columns = churn_df.select_dtypes(include="object").columns.tolist()

encoder = LabelEncoder()

for col in non_numeric_columns:
    churn_df[col] = encoder.fit_transform(churn_df[col])



majority_class = churn_df[churn_df['Churn'] == 0]
minority_class = churn_df[churn_df['Churn'] == 1]


majority_class_undersampled = majority_class.sample(n=minority_class.shape[0], random_state=42)


balanced_churn_df = pd.concat([majority_class_undersampled, minority_class])


balanced_churn_df = balanced_churn_df.sample(frac=1, random_state=42).reset_index(drop=True)


X = balanced_churn_df.drop("Churn", axis=1)
y = balanced_churn_df["Churn"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

models = [LogisticRegression(class_weight='balanced'), SVC()]

for model in models:
    name = model.__class__.__name__
    
    model.fit(X_train, y_train)
    print("Model: ", name)
    y_pred = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    pr_score = precision_score(y_test, y_pred)
    rc_score = recall_score(y_test, y_pred)
    f_score = f1_score(y_test, y_pred)
    cf_matrix = confusion_matrix(y_test, y_pred)   

    print("Accuracy: ", accuracy)
    print("Persition score: ", pr_score)
    print("Recall score: ", rc_score)
    print("F1 Score: ", f_score)
    print("-----------------------------")
    print("Confusion Matrix",cf_matrix)


C:\Users\baxti\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Model:  LogisticRegression
Accuracy:  0.7828877005347593
Persition score:  0.7515274949083504
Recall score:  0.82
F1 Score:  0.7842720510095643
-----------------------------
Confusion Matrix [[363 122]
 [ 81 369]]
Model:  SVC
Accuracy:  0.732620320855615
Persition score:  0.6923076923076923
Recall score:  0.8
F1 Score:  0.7422680412371134
-----------------------------
Confusion Matrix [[325 160]
 [ 90 360]]
